## Predicting SAP Sales & Distribution Benchmark Results from cint_rate_base2006 Results
In this notebook, we attempt to correlate SPEC cint_rate_base2006 with SAPS on the SAP Sales & Distribution 2-tier benchmark.

https://www.sap.com/dmc/exp/2018-benchmark-directory/#/sd

This is because there is a much larger volume of cint_rate_base2006 data, as compared to SAP SD2 data. If we accept the assumption that SAP SD2 is a reasonable approximation of an enterprise workload, and if we can find a good correlation between cint_rate_base2006 and SAP SD2, then we can size enterprise workloads directly using cint_rate_base2006 data, whch is quite abundant.

In a previous notebook, I've consolidated cint_rate_base2006 and cint_rate_base2017 data and normalized to cint_rate_base2006 to provide a larger sample set.

In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import validation_curve
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

In [ ]:
!wget -O export-sd.csv https://www.sap.com/dmc/exp/2018-benchmark-directory/assets/export-sd.csv

In [ ]:
dfsaps = pd.read_csv("export-sd.csv", sep=";", error_bad_lines=False, header=0, encoding = "ISO8859-1")

dfsaps['Server Name'] = dfsaps['Server Name'].str.upper() 
dfsaps['CPU Architecture'] = dfsaps['CPU Architecture'].str.upper() 
dfsaps['CPU Speed'] = dfsaps['CPU Speed'].str.upper() 
dfsaps['Technology Partner'] = dfsaps['Technology Partner'].str.upper() 

# we should only get 2-tier results
dfsaps = dfsaps[ dfsaps['Configuration'] == '2-tier' ]

dfsaps = dfsaps.drop(dfsaps.columns[[8, 12, 13, 14, 22, 24]], axis=1)

# unfortunately some of the benchmarks have incorrect core counts
# e.g. 2005021 has a null value for Cores in the CSV (but the long description shows 32 processors)
# so we just drop any entries where Cores is not defined

dfsaps.dropna(subset=['Cores'], inplace=True)
dfsaps.to_csv("saps.csv", index=False)

dfsaps

## Attempt to Derive Correlation Between SAPS and SPEC
SAPS is a whole-system, complex benchmark, which is more relevant for enterprise workloads. However, the number of available SAPS benchmarks is low and mostly biased towards large, high end systems.

If we can find a strong correlation between SAPS and SPEC, then we can use SPEC as a proxy for estimating performance of different processor architectures on SAPS-like, enterprise workloads.

Rather than manually looking up cint_rate_base2006 values for every entry in the SAPS SD2 benchmark, we first attempt to automatically match them; because cint_rate_base2006 benchmarks don't break out the processor version, cores, and clock speed as separate fields in the summary, we have to do some parsing. To get a better match, we only use SAPS with "INTEL XEON" in the CPU Architecture description. This is 296 entries which is a little less than half of the total entries.

In [ ]:
dfintel = dfsaps[ (dfsaps['CPU Architecture'].str.contains(r'^INTEL XEON')) ]

dfintel

Load the SPEC ratings, and extract only those for INTEL XEON where a clock speed is specified in SYSTEM NAME

In [ ]:
dfspec = pd.read_csv("specrate.csv")

# only get the SPEC results for INTEL XEON which have a clock speed (so we can match it to the SAPS dataframe)
dfspecintel = dfspec[ (dfspec['SYSTEM NAME'].str.contains('\(INTEL XEON'))]
dfspecintel = dfspecintel[ (dfspecintel['SYSTEM NAME'].str.contains('GHZ'))]

dfspecintel.head(10)

## Correlating Technology Partner
These are the top Technology Partner submissions in the SAPS Intel Xeon submissions

In [ ]:
dfintel.groupby("Technology Partner")["Certification Number"].count().head(10).sort_values(ascending=False)

And these are the top TEST SPONSOR submissions in SPEC. We have to "fix" these so that a join is possible.

In [ ]:
dfspecintel.groupby("TEST SPONSOR")["RESULTS BASE 2006"].count().head(10).sort_values(ascending=False)

In [ ]:
dfspecintel = dfspecintel.replace(to_replace='DELL INC.', value='DELL', regex=False)
dfspecintel = dfspecintel.replace(to_replace='DELL INC', value='DELL', regex=False)
dfspecintel = dfspecintel.replace(to_replace='BULL SAS', value='BULL', regex=False)

In [ ]:
dfspecintel.groupby("TEST SPONSOR")["RESULTS BASE 2006"].count().head(10).sort_values(ascending=False)

Iterate over the "Intel Xeon" SAPS dataframe and filter the Intel SPEC dataframe by Technology Partner / TEST SPONSOR, Cores / PROCESSOR ENABLED CORES, Server Name / SYSTEM NAME (substring), CPU Speed / SYSTEM NAME (substring).

When we only match the Xeon model number, core count, and clock speed, we get a good number of matches but there's a lot of duplication. We should only keep the unique entries from the SAPS dataframe (i.e. Certification Number).



In [ ]:
import re

pd.set_option('display.max_colwidth', None)
pd.set_option('mode.chained_assignment', None)

c = 0
d = 0
e = 0

# 3 or more digits.. assume this is the Xeon model number
r1 = re.compile('\s+([a-zA-Z0-9_-]*\d{3,})')

# assume this is clock speed
r2 = re.compile('(\d+\.\d+)\s?GHZ')

newdf = pd.DataFrame()
newdf2 = pd.DataFrame()

while (c < len(dfintel.index)):
    
    row = dfintel.iloc[c]
    tech_partner = row['Technology Partner']
    server_name =  row['Server Name']
    cpu_arch = row['CPU Architecture']
    cpu_speed = row['CPU Speed']
    cores = row['Cores']
    certnum = row['Certification Number']
    certdate = row['Certification Date']
    saps = row['saps']


    # for CPU architecture we just want to extract the 4-digit Xeon model number
    m = r1.search( cpu_arch )
    if m:
        model = m.group(1)

    # for CPU speed, we want to get rid of the GHZ bit
    m = r2.match( cpu_speed )
    if m:
        clock_speed = m.group(1)

    if ((len(model) > 0) & (len(clock_speed))) > 0:
        # just match Xeon model number, cores, clock speed
        res = dfspecintel[ (
            (dfspecintel['PROCESSOR ENABLED CORES'] == cores) &
            (dfspecintel["SYSTEM NAME"].str.contains(clock_speed)) &
            (dfspecintel["SYSTEM NAME"].str.contains(model)) 
        )]
        
        if len(res.index) > 0:
            res["Technology Partner"] = tech_partner
            res["Server Name"] = server_name
            res["CPU Architecture"] = cpu_arch
            res["CPU Speed"] = cpu_speed
            res["Cores"] = cores
            res['Certification Number'] = certnum
            res['Certification Date'] = certdate
            res['SAPS'] = saps
            
            newdf = newdf.append(res)
            d = d + 1

    c = c + 1

print('%d out of %d matched' % (d, c))
newdf = newdf.drop_duplicates(subset=['Certification Number'], keep='first')

newdf.sort_values(['Certification Number'], ascending=True)

In [ ]:
# save the correlated values
newdf.to_csv("correlated_base.csv", index=False)

# cleaned-up for manual sanity-checking
newdf_clean = newdf.copy()
newdf_clean = newdf_clean.drop(newdf_clean.columns[[0, 5, 6, 10, 11]], axis=1)

newdf_clean

In [ ]:
newdf_clean['SAPS'].describe()

In [ ]:
newdf_clean['RESULTS BASE 2006'].describe()

While correlation of SAPS and cint_rate_base2006 is very good, a linear fit won't be accurate for very small or very large machines; small machines have proportionately higher SAPS per cint_rate_base2006 (up to 140+) while large systems have proportionately lower SAPS per cint_rate_base2006 (around 50).

Also, the lowest reported SAPS for the Intel Xeon machines is around 10000, but there are older/smaller systems that need to be considered.

In [ ]:
newdf_clean['SAPS'].corr(newdf['RESULTS BASE 2006'])

## Polynomial Fitting
We evaluate a polynomial fit for better predictive accuracy. The RISC-only manually labeled results are not very accurate by themselves, so we use the consolidated results. Let's first reshape the data into a useful format.

In [ ]:
# get the automatically-labeled Intel results
d2 = newdf_clean[['RESULTS BASE 2006', 'SAPS' ]].copy()

d2['RESULTS BASE 2006'] = pd.to_numeric(d2['RESULTS BASE 2006'], errors='coerce')
d2['SAPS'] = pd.to_numeric(d2['SAPS'], errors='coerce')

## FIXME: remove entries where the SPEC is > 10000
d2 = d2[ (d2['RESULTS BASE 2006'] < 10000) ]

d2.to_csv("saps_to_spec_correlated_vals.csv", index=False)

In [ ]:
X = d2.iloc[:, 0].values.reshape(-1, 1)
y = d2.iloc[:, 1].values.reshape(-1, 1)

X_seq = np.linspace(X.min(),X.max(),len(d2)).reshape(-1,1)

Now let's try to find the polynomial degree which will result in the best fit.

In [ ]:
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                         LinearRegression(**kwargs))

degree = np.arange(0, 7)
train_score, val_score = validation_curve(PolynomialRegression(), X, y,
                                          'polynomialfeatures__degree', degree, cv=7)

plt.plot(degree, np.median(train_score, 1), color='blue', label='training score')
plt.plot(degree, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('degree')
plt.ylabel('score');

It looks like degree=1 gives the best fit, so a polynomial fit is not required at all, linear regression would do fine. However, to more accurately model the outliers, an increased degree may be desirable. We do note that when degree > 5 overfitting already occurs.

In [ ]:
degree=1

polyreg=make_pipeline(PolynomialFeatures(degree),LinearRegression())
polyreg.fit(X,y)

plt.figure()
plt.scatter(X,y)
plt.plot(X_seq,polyreg.predict(X_seq),color="black")
plt.title("Polynomial regression with degree "+str(degree))
plt.show()

c = np.polyfit(d2['RESULTS BASE 2006'], d2['SAPS'], degree)
print(c)

## Ensemble Regressors
A RandomForestRegressor results in better results than the GradientBoostingRegressor.

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn import ensemble, neural_network
from sklearn.metrics import mean_squared_error, r2_score

X = d2.iloc[:, 0].values.reshape(-1, 1)
y = d2.iloc[:, 1].values.reshape(-1, 1)
y = np.ravel(y)

x_training_set, x_test_set, y_training_set, y_test_set = train_test_split(X,y,test_size=0.20, 
                                                                          random_state=42,
                                                                          shuffle=True)
# GradientBoostingRegressor
#params = {'n_estimators': 500, 'max_depth': 5, 'min_samples_split': 2,
#          'learning_rate': 0.01, 'loss': 'huber', 'validation_fraction': 0.2,
#            'n_iter_no_change': 5, 'tol': 0.0001 }
#model = ensemble.GradientBoostingRegressor(**params)

# this doesn't converge
#params = { 'learning_rate': 'adaptive' }
#model = neural_network.MLPRegressor(**params)

params = {'n_estimators': 500, 'max_depth': None, 'min_samples_split': 2, 'criterion': 'mse' }
model = ensemble.RandomForestRegressor(**params)

model.fit(x_training_set, y_training_set)

In [ ]:
model_score = model.score(x_training_set,y_training_set)

print('R2 sq: ', model_score)
y_predicted = model.predict(x_test_set)

print("Mean Squared Error: %.2f"% mean_squared_error(y_test_set, y_predicted))

# Explained variance score: 1 is perfect prediction
print('Test Variance Score: %.2f' % r2_score(y_test_set, y_predicted))

Let's save our model so that we don't need to go through the pain and suffering of re-training again later.

In [ ]:
import pickle

pkl_filename = "cint_rate_base2006_to_saps.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

We clearly see in the graph below that this model has a huge error for a large valued data point (actual is 800K+ and predicted is less than 600K). This is a single data point (the Fujitsu M10-4S).  As new SAPS and SPEC ratings come up for newer and larger machines, this model will have to be re-trained

The most conservative way to approach this problem is to simply reject any queries where the cint_rate_base2006 is more than 10000 or so.

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_test_set, y_predicted, edgecolors=(0, 0, 0))
ax.plot([y_test_set.min(), y_test_set.max()], [y_test_set.min(), y_test_set.max()], 'k--', lw=4)
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
ax.set_title("Ground Truth vs Predicted")
plt.show()

In [ ]:
# formula for SAPS from cint_rate_base2006
def spec2saps(spec: float) -> float:
    i = 0
    saps = 0
    while i < len(c):
        p = (len(c) - 1) - i
        saps = saps + c[i] * (spec**p)
        i = i + 1
   
    return (round(saps,-1))

def spec2saps2(spec: float) -> float:
    a = np.array([spec])
    a = np.expand_dims(a, 0)
    saps = model.predict(a)[0]

    return (round(saps,-1))

## Analysis

Let's compare the polynomial fit with the ensemble model. Over the (small) manual validation set, the ensemble model provides qualitatively better results than the polynomial fit, except for the single very large sample (M10-4S).

In [ ]:
# Fujitsu M10-4S (836550 SAPS and 13625.00 cint_rate_base2006)
print (spec2saps(13625))
print (spec2saps2(13625))

In [ ]:
# SUN FIRE V490 (ULTRASPARC IV, 6750 SAPS, 71.70 cint_rate_base2006)
print (spec2saps(71.70))
print (spec2saps2(71.70))

In [ ]:
# Intel Xeon 7140M (10380 SAPS, 76.9 cint_rate_base2006)
print (spec2saps(76.9))
print (spec2saps2(76.9))

In [ ]:
# Sun M9000 (2.88GHz, 175600 SAPS and 2400 cint_rate_base2006)

print (spec2saps(2400))
print (spec2saps2(2400))

In [ ]:
# Sun M3000 (2.52GHz, 4130 SAPS and 25.7 cint_rate_base2006)
print (spec2saps(25.7))
print (spec2saps2(25.7))

In [ ]:
# IBM POWER 730, 47600 SAPS, 515 cint_rate_base2006, the error is large because the model is mostly influenced
# by Intel Xeon data points
print (spec2saps(515))
print (spec2saps2(515))

In [ ]:
# Here's an Intel Xeon data point.. good fit
# CISCO UCS C260 M2 (INTEL XEON E7-2870, 2.40 GHZ)  36600 SAPS, 526 cint_rate_base2006
print (spec2saps(526))
print (spec2saps2(526))

In [ ]:
# another Intel Xeon data point (a rather large system)
# CISCO UCS B200 M5 (INTEL XEON PLATINUM 8276, 2.20GHZ) 131170 SAPS, 2868.71 cint_rate_base2006
print (spec2saps(2868.71))
print (spec2saps2(2868.71))